# [Module 3] CNN-QR, DeepAR+ 및 Prophet Predictor 생성 
- 여기서는 세개의 Predictos 인 CNN-QR, Prophet 과 DeepAR+를 생성 (포케스팅 모델 학습) 을 하고, 실제 Predictor의 성능 지표를 통해서 얼마나 성능이 나왔는지를 학인 할 수 있습니다.
* 이 과정은 약 60분 정도 걸립니다. **About 60 mins may be elapsed**

---
2020.08 에 출신된 CNN-QR은 아래 블로그를 참조 하세요. <br>
Amazon Forecast can now use Convolutional Neural Networks (CNNs) to train forecasting models up to 2X faster with up to 30% higher accuracy
- https://aws.amazon.com/blogs/machine-learning/amazon-forecast-can-now-use-convolutional-neural-networks-cnns-to-train-forecasting-models-up-to-2x-faster-with-up-to-30-higher-accuracy/

In [2]:
import boto3
from time import sleep
import pandas as pd
import json
import time
import pprint
import numpy as np

In [3]:
# Recover variables 
%store -r

In [4]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')

## Parameters
- forecastHorizon 는 31 개의 미래 데이타 포인트를 예측 합니다. ForecastFrequency 가 Day 기준이므로 미래의 31일을 예측 합니다.
- NumberOfBacktestWindows 는 백테스트를 의미합니다. 학습 데이타의 맨 마지막에서 2번 백테스트 합니다.

In [5]:
forecastHorizon = 31 # the number of forecasting
NumberOfBacktestWindows = 2 # the number of Backtests
BackTestWindowOffset = 31 # the point from the end of the dataset where the data is split for model training and testing 
ForecastFrequency = "D"

![Fig.3.2.ForecastHorizon](img/Fig.3.2.ForecastHorizon.png)

![BackTest](img/Fig3.1.BackTest.png)

자세한 파라미터의 정의는 아래 링크를 클릭 해주세요.<br>
Detailed parameter info is [here](https://docs.aws.amazon.com/forecast/latest/dg/metrics.html)

## Predictor 생성

#### Predictor algorithms
- 아래 세개의 알고리즘을 사용 합니다.

In [6]:
cnnqqr_algorithmArn = 'arn:aws:forecast:::algorithm/CNN-QR'
prophet_algorithmArn = 'arn:aws:forecast:::algorithm/Prophet'
deepAR_Plus_algorithmArn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus'

### CNN-QR
- CNN-QR을 학습하기 위해서 필요한 설정 파일들을 기술 합니다. 
- 시계열 데이터가 결측일 경우에 채우는 전략은 아래 링크를 참조 하세요.
    - [FeaturizationMethod](https://docs.aws.amazon.com/forecast/latest/dg/API_FeaturizationMethod.html)
    - 여기서는 아래와 같이 결측 설정을 하였습니다.
    ```
   "FeaturizationMethodParameters": 
    {"frontfill": "none", # 시계열 시작 지점 부터의 과거는 고려 하지 않음
     "middlefill": "nan", # 시계열 시작과 끝 지점의 중간 데이터의 미싱은 무시하고 채우지 않음
     "backfill": "nan"} # 시계열 끝 지점 이후의 미래는 고려 하지 않음
    
    ```

In [7]:
cnnqr_predictor_name= project+'_cnnqr_' + target_suffix + suffix

In [10]:
# Build cnnqr:
cnnqr_create_predictor_response=forecast.create_predictor(
      PredictorName = cnnqr_predictor_name, 
      AlgorithmArn = cnnqqr_algorithmArn,
      ForecastHorizon= forecastHorizon,
      PerformAutoML= False,
      PerformHPO= False,
      EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                             "BackTestWindowOffset": BackTestWindowOffset}, 
      InputDataConfig= {"DatasetGroupArn": target_datasetGroupArn, 
                        "SupplementaryFeatures": [ 
                         { 
                            "Name": "holiday",
                            "Value": "US"
                         }
                      ]},
      FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                            "ForecastDimensions":
                            ["store"],
                            "Featurizations": 
                            [
                              {"AttributeName": "target_value", 
                               "FeaturizationPipeline": 
                                [
                                  {"FeaturizationMethodName": "filling", 
                                   "FeaturizationMethodParameters": 
                                    {"frontfill": "none", 
                                     "middlefill": "nan",
                                     "backfill": "nan"} 
                                  }
                                ]
                              }
                            ]
                           }
)
                                                 

### Prophet
- Prophet을 학습하기 위해서 필요한 설정 파일들을 기술 합니다. 

In [12]:
# Prophet Specifics
prophet_predictorName= project+'_prophet_algo_1' + target_suffix + suffix

In [14]:
# Build Prophet:
prophet_create_predictor_response=forecast.create_predictor(
      PredictorName=prophet_predictorName, 
      AlgorithmArn=prophet_algorithmArn,
      ForecastHorizon=forecastHorizon,
      PerformAutoML= False,
      PerformHPO=False,
      EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                             "BackTestWindowOffset": BackTestWindowOffset}, 
      InputDataConfig= {"DatasetGroupArn": target_datasetGroupArn, 
                        "SupplementaryFeatures": [ 
                         { 
                            "Name": "holiday",
                            "Value": "US"
                         }
                      ]},
      FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                            "ForecastDimensions":
                            ["store"],
                            "Featurizations": 
                            [
                              {"AttributeName": "target_value", 
                               "FeaturizationPipeline": 
                                [
                                  {"FeaturizationMethodName": "filling", 
                                   "FeaturizationMethodParameters": 
                                    {"frontfill": "none", 
                                     "middlefill": "nan", 
                                     "backfill": "nan"}
                                  }
                                ]
                              }
                            ]
                           }
)                     

### DeepAR+

In [15]:
# DeepAR+ Specifics
deeparp_predictorName= project+'_deeparp_algo_1' + suffix

In [16]:
# Build DeepAR+:
deeparp_create_predictor_response=forecast.create_predictor(
      PredictorName=deeparp_predictorName, 
      AlgorithmArn=deepAR_Plus_algorithmArn,
      ForecastHorizon=forecastHorizon,
      PerformAutoML= False,
      PerformHPO=False,
      EvaluationParameters= {"NumberOfBacktestWindows": NumberOfBacktestWindows, 
                             "BackTestWindowOffset": BackTestWindowOffset}, 
      InputDataConfig= {"DatasetGroupArn": target_datasetGroupArn, "SupplementaryFeatures": [ 
                         { 
                            "Name": "holiday",
                            "Value": "US"
                         }
                      ]},
      TrainingParameters = {
                            "likelihood": "negative-binomial",
                            "num_averaged_models": "5"
                            },    
      FeaturizationConfig= {"ForecastFrequency": ForecastFrequency, 
                            "ForecastDimensions":
                            ["store"],
                            
                            "Featurizations": 
                            [
                              {"AttributeName": "target_value", 
                               "FeaturizationPipeline": 
                                [
                                  {"FeaturizationMethodName": "filling", 
                                   "FeaturizationMethodParameters": 
                                    {"frontfill": "none", 
                                     "middlefill": "nan", 
                                     "backfill": "nan"}
                                  }
                                ]
                              }
                            ]
                           }
)                           

## Check if the predictors are active
- Predictor가 실제로 생성되고 있는 상태를 확인 합니다.

In [17]:
target_cnnqr_predictorArn = cnnqr_create_predictor_response["PredictorArn"]
target_prophet_predictorArn = prophet_create_predictor_response["PredictorArn"]
target_deepar_predictorArn = deeparp_create_predictor_response["PredictorArn"]

<font color="red">아래 셀의 [*] 가 숫자로 바뀔때까지 기다려 주세요.</font>
**약 60분 걸립니다.**

In [18]:
%%time
# Check the Prophet status

while True:
    createProphetPredictorStatus = forecast.describe_predictor(PredictorArn= target_prophet_predictorArn)['Status']
    createDeeparpPredictorStatus = forecast.describe_predictor(PredictorArn= target_deepar_predictorArn)['Status']    
    createCnnqrPredictorStatus = forecast.describe_predictor(PredictorArn= target_cnnqr_predictorArn)['Status']        
    print("Prophet: ", createProphetPredictorStatus)
    print("DeepARP: ", createDeeparpPredictorStatus)
    print("Cnnqr: ", createCnnqrPredictorStatus)    
    if createProphetPredictorStatus != 'ACTIVE' and createProphetPredictorStatus != 'CREATE_FAILED':
        sleep(60)
    elif createDeeparpPredictorStatus != 'ACTIVE' and createDeeparpPredictorStatus != 'CREATE_FAILED':
        sleep(60)
    elif createCnnqrPredictorStatus != 'ACTIVE' and createCnnqrPredictorStatus != 'CREATE_FAILED':
        sleep(60)        
    else:
        break

Prophet:  CREATE_PENDING
DeepARP:  CREATE_PENDING
Cnnqr:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Cnnqr:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Cnnqr:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Cnnqr:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Cnnqr:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Cnnqr:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Cnnqr:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Cnnqr:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Cnnqr:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Cnnqr:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Cnnqr:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Cnnqr:  CREATE_

## Predictor evaluation
-아래 화면은 Forecast Console --> Dataset Group 선택 --> Predictors 선택을 하시면 조금 더 쉽게 볼 수 있습니다. 
![PredictorMetric](img/Fig.3.2.PredictorMetric.png)

#### CNN-QR

In [19]:
cnnqr_metrics = forecast.get_accuracy_metrics(PredictorArn=target_cnnqr_predictorArn)
pp = pprint.PrettyPrinter()
pp.pprint(cnnqr_metrics)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/CNN-QR',
                                 'TestWindows': [{'EvaluationType': 'SUMMARY',
                                                  'Metrics': {'RMSE': 9.916570754369154,
                                                              'WeightedQuantileLosses': [{'LossValue': 0.06506357591989097,
                                                                                          'Quantile': 0.9},
                                                                                         {'LossValue': 0.12748531583719175,
                                                                                          'Quantile': 0.5},
                                                                                         {'LossValue': 0.05111708176177882,
                                                                                          'Quantile': 0.1}]}},
                                             

#### Prophet

In [20]:
# Prophet Metrics
# prophet_arn = prophet_create_predictor_response['PredictorArn']
prophet_metrics = forecast.get_accuracy_metrics(PredictorArn=target_prophet_predictorArn)
pp = pprint.PrettyPrinter()
pp.pprint(prophet_metrics)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Prophet',
                                 'TestWindows': [{'EvaluationType': 'SUMMARY',
                                                  'Metrics': {'RMSE': 8.507449156522501,
                                                              'WeightedQuantileLosses': [{'LossValue': 0.051280372916466574,
                                                                                          'Quantile': 0.9},
                                                                                         {'LossValue': 0.10972941054792003,
                                                                                          'Quantile': 0.5},
                                                                                         {'LossValue': 0.04660276759016778,
                                                                                          'Quantile': 0.1}]}},
                                           

#### DeepAR+

In [21]:
# DeepAR+ Metrics
# deepar_predictor_arn = deeparp_create_predictor_response['PredictorArn']
deeparp_metrics = forecast.get_accuracy_metrics(PredictorArn=target_deepar_predictorArn)
pp = pprint.PrettyPrinter()
pp.pprint(deeparp_metrics)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Deep_AR_Plus',
                                 'TestWindows': [{'EvaluationType': 'SUMMARY',
                                                  'Metrics': {'RMSE': 10.195393881125298,
                                                              'WeightedQuantileLosses': [{'LossValue': 0.0619224906192548,
                                                                                          'Quantile': 0.9},
                                                                                         {'LossValue': 0.13128978163976018,
                                                                                          'Quantile': 0.5},
                                                                                         {'LossValue': 0.05450154571469425,
                                                                                          'Quantile': 0.1}]}},
                                       

In [22]:
%store target_prophet_predictorArn
%store target_deepar_predictorArn
%store target_cnnqr_predictorArn

Stored 'target_prophet_predictorArn' (str)
Stored 'target_deepar_predictorArn' (str)
Stored 'target_cnnqr_predictorArn' (str)
